In [ ]:
# leave for imports

Welcome to Arctic Fox! What is it?

Arcitc Fox is a code autoamtion tool for reusable, sharable, and adjustable automations! In the sandbox you can experiment with automations we've created and also create your own automations!

# What's an Automation?

As with all software development we'll start with a Hello World example! You can see the HelloWorld automation below.

Run Arctic Fox to see the HelloWorld automation produce code. Click the Arctic Fox icon on the left and press Automate in the Arctic Fox pane.

In [ ]:
#[HelloWorld]

What about sending parameters into an automation? Parameters can be passed in through position or named parameters. Change "YourName" below to your name. The two below are identical.

In [ ]:
#[HelloWorld YourName]

#[HelloWorld --name YourName]

And lastly, for the intro, you can easily removed automated code using the scrub button. In the Arctic Fox pane, click Scrub!

And Boom! The automated code has vanished! Move on to the next step to keep learning about Arctic Fox!

# How is an Automation in Code?

Automations are placed in a comments so it's presence doesn't impact your code base! We also give it a little formatting. The automation is enclosed in light gray brackets and the autoamtion name is bolded. The HelloWorld automation is equal to the following text:

#\[HelloWorld\]

#\[HelloWorld YourName\]

#\[HelloWorld --name YourName\]

### Continue to the next step to move past Hello World!